In [10]:
from typing import List, Dict, Union

import simpy
import pandas as pd
import pybamm

pybamm.set_logging_level("NOTICE")

In [11]:
class SimpleBattery:

    def __init__(self, capacity, soc=0):
        self.capacity = capacity
        self.soc = soc

    def update(self, energy):
        self.soc += energy
        excess_energy = 0

        if self.soc < 0:
            excess_energy = self.soc
            self.soc = 0
        elif self.soc > self.capacity:
            excess_energy = self.soc - self.capacity
            self.soc = self.capacity

        return excess_energy

In [12]:
class PybammBattery:

    def __init__(self, env, capacity, soc=0):
        """
        'Nominal cell capacity [A.h]' is set twice the capacity provided, so that we can easily calculate the excess power.
        Otherwise the excess energy cannot be calculated since the battery won't charge pass the capacity.
        """
        self.env = env
        self.capacity = capacity
        self.soc = soc
        self.excess_power = 0
        self.experiment = None
        
        self.last_solution = None

        self.parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Chen2020)
        self.model = pybamm.lithium_ion.DFN()

    def update(self, power):
        if power == 0:
            return 0;
        
        self.create_experiment(power)

        sim = pybamm.Simulation(self.model, parameter_values=self.parameter_values, experiment=self.experiment)
        self.last_solution = sim.solve(starting_solution=self.last_solution)
        discharge_capacity = self.last_solution['Discharge capacity [A.h]']

        self.calculate_state_of_charge(discharge_capacity.entries[-1])
    
    # Not sure if the logic in this method is right, but what I am trying to do is to calculate the excess power
    def calculate_excess_energy(self):
        self.excess_power = (self.soc * self.capacity) - self.capacity
        
    def calculate_state_of_charge(self, current_discharge_or_charge):
        self.soc = current_discharge_or_charge;
        
    def create_experiment(self, power):
        # So the power is always in watts. Not sure how to charge the battery with this power.
        # Due to this reason the following logic is implemented, which is probably not right
        if power > 0:
            charge_or_discharge = f'charge at {0.1 * power} W for 15 s'
        else:
            charge_or_discharge = f'discharge at {-0.1 * power} W for 15 s'
            
        
        self.experiment = pybamm.Experiment([charge_or_discharge])

In [13]:
def simulate(env: simpy.Environment,
             battery: Union[SimpleBattery, PybammBattery],
             power_delta_list: List[float],
             records: List[Dict]):
    
    for power_delta in power_delta_list:
        yield env.timeout(1)
        battery.update(power_delta)
        records.append({
            "power_delta": power_delta,
            "excess_power": battery.excess_power,
            "soc": battery.soc,
            "capacity [A.h]": battery.capacity
        })

In [14]:
# For now let's assume the simple case of one step every second where we first (dis)charge and then implicitly read.
# Later we can extend this to a more asynchronous charge/discharge/read pattern with different processes if we want
power_delta_list = [1, -3, 2, 3, 4, -2]
records = []  # log of some infos for later analysis

env = simpy.Environment()
battery = PybammBattery(env, capacity=5)
env.process(simulate(env, battery, power_delta_list, records))
env.run()

result = pd.DataFrame(records)
with open("result.csv", "w") as f:
    f.write(result.to_csv())
print(result)

2022-03-31 10:23:17,322 - [NOTICE] simulation.solve(855): Cycle 1/1 (25.267 ms elapsed) --------------------
2022-03-31 10:23:17,322 - [NOTICE] simulation.solve(889): Cycle 1/1, step 1/1: charge at 0.1 W for 15 s
2022-03-31 10:23:17,586 - [NOTICE] simulation.solve(1011): Finish experiment simulation, took 290.089 ms
2022-03-31 10:23:18,146 - [NOTICE] simulation.solve(855): Cycle 2/2 (25.586 ms elapsed) --------------------
2022-03-31 10:23:18,146 - [NOTICE] simulation.solve(889): Cycle 2/2, step 1/1: discharge at 0.30000000000000004 W for 15 s
2022-03-31 10:23:18,401 - [NOTICE] simulation.solve(1011): Finish experiment simulation, took 279.995 ms
2022-03-31 10:23:18,850 - [NOTICE] simulation.solve(855): Cycle 3/3 (25.603 ms elapsed) --------------------
2022-03-31 10:23:18,850 - [NOTICE] simulation.solve(889): Cycle 3/3, step 1/1: charge at 0.2 W for 15 s
2022-03-31 10:23:19,104 - [NOTICE] simulation.solve(1011): Finish experiment simulation, took 279.339 ms
2022-03-31 10:23:19,552 - [

   power_delta  excess_power           soc  capacity [A.h]
0            1             0 -9.962749e-05               5
1           -3             0  1.996090e-04               5
2            2             0  3.905016e-07               5
3            3             0 -2.982989e-04               5
4            4             0 -6.963708e-04               5
5           -2             0 -4.969959e-04               5
